Шаг 0. Установка необходимых библиотек и модулей.

In [ ]:
!pip install requests

!pip install beautifulsoup4

In [ ]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd

Шаг 1. Создание списка страничек сайта Mental Floss, с которых будут парситься ссылки на отдельные статьи.

In [ ]:
pages = []
page_numbers = list(range(1,11))
link = "https://www.mentalfloss.com/section/science?page="

In [ ]:
for i in page_numbers:
  pages.append(link+str(i))

Шаг 2. Добавление ссылок на отдельные статьи с собранных страниц в отдельный список.

In [ ]:
links = []

In [ ]:
for i in pages:
  url = i
  page = rq.get(i)
  soup = BeautifulSoup(page.text, features = "html.parser")
  for x in soup.find_all("a"):
    if x.parent.name == "article":
      links.append(x.get('href'))

In [ ]:
print(links)
print(len(links))

['https://www.mentalfloss.com/posts/easter-island-study-contradicts-ecocide', 'https://www.mentalfloss.com/posts/misconceptions-about-weather', 'https://www.mentalfloss.com/posts/forensic-kit-murder-bag-origins', 'https://www.mentalfloss.com/posts/solar-storms-northern-lights-facts', 'https://www.mentalfloss.com/posts/crows-count-out-loud', 'https://www.mentalfloss.com/posts/t-rex-juvenile-fossil', 'https://www.mentalfloss.com/article/654695/people-who-operated-themselves', 'https://www.mentalfloss.com/posts/joro-spider-invasive-species', 'https://www.mentalfloss.com/posts/woolly-mammoth-bones-in-wine-cellar', 'https://www.mentalfloss.com/article/90259/6-bizarre-sharks-live-deep-ocean', 'https://www.mentalfloss.com/posts/having-anger-issues-try-shredding-your-feelings', 'https://www.mentalfloss.com/posts/never-park-under-overpass-during-tornado-storm', 'https://www.mentalfloss.com/posts/can-dead-body-get-suntan', 'https://www.mentalfloss.com/posts/why-beer-tastes-better-cold', 'https:/

Шаг 3. Создание датафрейма из ссылок, текстов статей и тегов к ним.

In [ ]:
titles = []
tags = []
texts = []

In [ ]:
def parseparse(x):
  for i in x:
    article_url = i
    article_rq = rq.get(article_url)
    article_soup = BeautifulSoup(article_rq.text, features = "html.parser")

    for title in article_soup.find_all("title"):
      if title.parent.name == "head":
        titles.append(title.text)

    article_tags = article_soup.find_all("meta", {"property" : "article:tag"})
    tag_set = []
    for tag in article_tags:
      tag_set.append(tag.get("content"))
    tags.append(tag_set)

    article_text = article_soup.find_all("p")
    paragraphs = []
    for text in article_text:
      paragraphs.append(text.text)
    texts.append(paragraphs)

  df0 = pd.DataFrame(titles, columns = ["titles"])
  df0["tags"] = tags
  df0["texts"] = texts

  return df0

In [ ]:
df = parseparse(links)

In [ ]:
df.shape

(240, 3)

In [ ]:
df["links"] = links

In [ ]:
df

,titles,tags,texts,links
0,Rock Gardens Contradict Easter Island âEcoci...,"[science, history, archaeology, mystery, news,...",[When British explorer James Cook arrived at R...,https://www.mentalfloss.com/posts/easter-islan...
1,Wild Misconceptions About Weather,"[weather, misconceptions, YouTube]",[If asked to grab a box of Crayola crayons and...,https://www.mentalfloss.com/posts/misconceptio...
2,The Grisly 1924 Crime That Led to the Forensic...,"[history, crime, murder, 2024SEOnew]","[In the United States, the term murder bag has...",https://www.mentalfloss.com/posts/forensic-kit...
3,12 Stunning Facts About Solar Storms,"[space, weather, science]","[A solar storm struck Earth in May 2024, deliv...",https://www.mentalfloss.com/posts/solar-storms...
4,Counting Crows: These Smart Birds Can Count Ou...,"[birds, crows, science, news]",[Calling someone a âbird brainâ isnât re...,https://www.mentalfloss.com/posts/crows-count-...
...,...,...,...,...
235,What is LakeâEffect Snow?,"[news, science, snowmen, weather]",[Parts of the northeastern United States are l...,https://www.mentalfloss.com/article/60154/what...
236,Why Brussels Sprouts Taste Better Than They Us...,"[food, plants, recipes, pop culture, vegetable...",[The fourth season of Leave It to Beaver opens...,https://www.mentalfloss.com/posts/do-brussels-...
237,"The 50 Most Beautiful Baby Names, According to...","[lists, babies, science, news]",[Naming a baby is one of the most fraught expe...,https://www.mentalfloss.com/posts/most-beautif...
238,10 Facts About Wisdom Teeth,"[biology, evolution, health, news, science, te...","[Like puberty and your first heartbreak, the p...",https://www.mentalfloss.com/article/554979/fac...


In [ ]:
df.to_csv('sci-fi.csv', encoding = "utf-8")